In [1]:
import math
import tensorflow as tf
import numpy as np
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import os
from tensorflow.contrib import rnn


tf.reset_default_graph()
tf.set_random_seed(777)

if "DISPLAY" not in os.environ:
    mpl.use('TkAgg')

import matplotlib.pyplot as plt

def MinMaxScaler(data):
    numerator = data - np.min(data,0)
    denominator = np.max(data,0)-np.min(data,0)
    return numerator / (denominator+1e-7)

xy = np.genfromtxt('/Users/yeseo/Desktop/201709.txt',delimiter = ',',dtype = None)
xy_with_noise = np.genfromtxt('/Users/yeseo/Desktop/eliminated_1.txt',delimiter = ',',dtype = np.float32)

xy= xy[:,:27]
a = xy[:,:2]
b = xy[:,2:]
data_max = np.max(b,1)
data_min = np.min(b,1)
b= MinMaxScaler(b)
xy = np.hstack((a,b))

xy_with_noise = xy_with_noise[:,:27]
a_with_noise = xy_with_noise[:,:2]
b_with_noise = xy_with_noise[:,2:]
b_with_noise = MinMaxScaler(b_with_noise)
xy_with_noise = np.hstack((a_with_noise,b_with_noise))



seq_length =6
data_dim =27
hidden_dim = 25
output_dim = 25
learning_rate = 0.01
iterations = 1000
epsilon = 1
e = math.exp(epsilon)
q = (1/(e+1))

train_size = int(len(xy)*0.7)


train_set = xy[:train_size]
test_set = xy[train_size:]

train_set_with_noise = xy_with_noise[:train_size]
test_set_with_noise = xy_with_noise[train_size:]

def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range (0,len(time_series)-seq_length):
        _x = time_series[i:i + seq_length, :]
        _y = time_series[i+seq_length,2:]
     
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

#train_set, test_set 만들기
trainX, trainY = build_dataset(train_set,seq_length)
testX,testY = build_dataset(test_set, seq_length)

trainX_with_noise, trainY_with_noise = build_dataset(train_set_with_noise,seq_length)
testX_with_noise,testY_with_noise = build_dataset(test_set_with_noise, seq_length)


X1 = tf.placeholder(tf.float32,[None, seq_length,data_dim])
Y1 = tf.placeholder(tf.float32,[None,25])

X2 = tf.placeholder(tf.float32,[None, seq_length,data_dim])
Y2 = tf.placeholder(tf.float32,[None,25])

#LSTM CELL만들기
with tf.variable_scope("rnn1"):
    cell1 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
   # multi_cell1 = tf.contrib.rnn.MultiRNNCell([cell1]*2)
    outputs1,_states1 = tf.nn.dynamic_rnn(cell1,X1,dtype = tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs1[:,-1], output_dim,activation_fn = None)

    loss1 =tf.reduce_mean(tf.square(Y_pred-Y1))
    train1 = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss1)


with tf.variable_scope("rnn2"):
    cell2 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    #multi_cell2 =  tf.contrib.rnn.MultiRNNCell([cell2]*2)
    outputs2,_states2 = tf.nn.dynamic_rnn(cell2, X2, dtype = tf.float32)
    Y_pred_with_noise = tf.contrib.layers.fully_connected(outputs2[:,-1], output_dim,activation_fn = None)

    loss2 =tf.reduce_mean(tf.square(Y_pred_with_noise-Y2))
    train2 = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss2)




#RMSE 측정
targets = tf.placeholder(tf.float32,[None,25])
predictions = tf.placeholder(tf.float32,[None,25])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets-predictions)))


x1 = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25])*2
x2 = x1+0.33*2
x3 = x2+0.33*2

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(iterations):
        _, step_loss1 = sess.run([train1,loss1],feed_dict={X1:trainX, Y1:trainY})
        print("[step: {}] loss: {}".format(i,step_loss1))
        _, step_loss2 = sess.run([train2,loss2],feed_dict={X2:trainX_with_noise, Y2:trainY_with_noise})
        print("[nstep: {}] loss: {}".format(i,step_loss2))
        
    test_predict = sess.run(Y_pred, feed_dict = {X1:testX})
    test_predict_with_noise = sess.run(Y_pred_with_noise, feed_dict = {X2:testX_with_noise})

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
[step: 0] loss: 0.33690133690834045
[nstep: 0] loss: 0.27755868434906006
[step: 1] loss: 0.2272791713476181
[nstep: 1] loss: 0.17710557579994202
[step: 2] loss: 0.16273143887519836
[nstep: 2] loss: 0.11819535493850708
[step: 3] loss: 0.11887208372354507
[nstep: 3] loss: 0.0813722237944603
[step: 4] loss: 0.08529619127511978
[nstep: 4] loss: 0.058974817395210266
[step: 5] loss: 0.06094425544142723
[nstep: 5] loss: 0.046768128871917725
[step: 6] loss: 0.04634390398859978
[nstep: 6] loss: 0.04079196974635124
[step: 7] loss: 0.03922262787818909
[nstep: 7] loss: 0.037346355617046356
[step: 8] loss: 0.03647897019982338
[nstep: 8] loss: 0.03554962947964668
[step: 9] loss: 0.0348692461848259
[nstep: 9] loss: 0.03402063250541687
[step: 10] loss: 0.03386818990111351
[

[nstep: 106] loss: 0.010906473733484745
[step: 107] loss: 0.002630656585097313
[nstep: 107] loss: 0.010934528894722462
[step: 108] loss: 0.00260296743363142
[nstep: 108] loss: 0.010889244265854359
[step: 109] loss: 0.002622848143801093
[nstep: 109] loss: 0.010834441520273685
[step: 110] loss: 0.0026493894401937723
[nstep: 110] loss: 0.010825123637914658
[step: 111] loss: 0.002651866525411606
[nstep: 111] loss: 0.010839611291885376
[step: 112] loss: 0.0026002952363342047
[nstep: 112] loss: 0.010836876928806305
[step: 113] loss: 0.0025471332482993603
[nstep: 113] loss: 0.010789221152663231
[step: 114] loss: 0.002511565573513508
[nstep: 114] loss: 0.010752665810286999
[step: 115] loss: 0.00250511197373271
[nstep: 115] loss: 0.010743997991085052
[step: 116] loss: 0.002520818030461669
[nstep: 116] loss: 0.010749725624918938
[step: 117] loss: 0.002539469860494137
[nstep: 117] loss: 0.010749060660600662
[step: 118] loss: 0.0025512734428048134
[nstep: 118] loss: 0.010723426938056946
[step: 119

[nstep: 217] loss: 0.009921672753989697
[step: 218] loss: 0.0017872316529974341
[nstep: 218] loss: 0.009792046621441841
[step: 219] loss: 0.0017854206962510943
[nstep: 219] loss: 0.009864702820777893
[step: 220] loss: 0.0017703853081911802
[nstep: 220] loss: 0.009876362048089504
[step: 221] loss: 0.001766537199728191
[nstep: 221] loss: 0.009771715849637985
[step: 222] loss: 0.0017693564295768738
[nstep: 222] loss: 0.00983146857470274
[step: 223] loss: 0.0017609840724617243
[nstep: 223] loss: 0.009836344048380852
[step: 224] loss: 0.0017505524447187781
[nstep: 224] loss: 0.00975063256919384
[step: 225] loss: 0.0017493994673714042
[nstep: 225] loss: 0.00979625154286623
[step: 226] loss: 0.0017490718746557832
[nstep: 226] loss: 0.009803978726267815
[step: 227] loss: 0.0017405395628884435
[nstep: 227] loss: 0.009731252677738667
[step: 228] loss: 0.0017325564986094832
[nstep: 228] loss: 0.00975851435214281
[step: 229] loss: 0.0017307507805526257
[nstep: 229] loss: 0.009775524958968163
[step

[step: 330] loss: 0.001370992511510849
[nstep: 330] loss: 0.00941670686006546
[step: 331] loss: 0.0013678792165592313
[nstep: 331] loss: 0.0092858225107193
[step: 332] loss: 0.001364513416774571
[nstep: 332] loss: 0.009682351723313332
[step: 333] loss: 0.0013617262011393905
[nstep: 333] loss: 0.009189569391310215
[step: 334] loss: 0.0013589703012257814
[nstep: 334] loss: 0.009365515783429146
[step: 335] loss: 0.00135550438426435
[nstep: 335] loss: 0.009427991695702076
[step: 336] loss: 0.0013515102909877896
[nstep: 336] loss: 0.009119225665926933
[step: 337] loss: 0.0013477103784680367
[nstep: 337] loss: 0.009421502240002155
[step: 338] loss: 0.0013443687930703163
[nstep: 338] loss: 0.009228816255927086
[step: 339] loss: 0.0013410791289061308
[nstep: 339] loss: 0.009163249284029007
[step: 340] loss: 0.0013375976122915745
[nstep: 340] loss: 0.009333423338830471
[step: 341] loss: 0.0013339727884158492
[nstep: 341] loss: 0.00909474492073059
[step: 342] loss: 0.0013305004686117172
[nstep: 

[step: 445] loss: 0.0010856181615963578
[nstep: 445] loss: 0.0086292065680027
[step: 446] loss: 0.0010833796113729477
[nstep: 446] loss: 0.008602794259786606
[step: 447] loss: 0.0010811787797138095
[nstep: 447] loss: 0.00862993486225605
[step: 448] loss: 0.0010789355728775263
[nstep: 448] loss: 0.008612634614109993
[step: 449] loss: 0.001076831598766148
[nstep: 449] loss: 0.00859070848673582
[step: 450] loss: 0.0010749292559921741
[nstep: 450] loss: 0.008607074618339539
[step: 451] loss: 0.0010733017697930336
[nstep: 451] loss: 0.008598385378718376
[step: 452] loss: 0.0010724423918873072
[nstep: 452] loss: 0.008580156601965427
[step: 453] loss: 0.0010733570670709014
[nstep: 453] loss: 0.00858445093035698
[step: 454] loss: 0.0010783830657601357
[nstep: 454] loss: 0.0085839768871665
[step: 455] loss: 0.0010935148457065225
[nstep: 455] loss: 0.008571616373956203
[step: 456] loss: 0.0011346859391778708
[nstep: 456] loss: 0.008565235882997513
[step: 457] loss: 0.001237856806255877
[nstep: 4

[step: 559] loss: 0.000899816513992846
[nstep: 559] loss: 0.008328796364367008
[step: 560] loss: 0.0008982880972325802
[nstep: 560] loss: 0.008246151730418205
[step: 561] loss: 0.0008967738831415772
[nstep: 561] loss: 0.008245603181421757
[step: 562] loss: 0.0008952891221269965
[nstep: 562] loss: 0.008316581137478352
[step: 563] loss: 0.0008938381215557456
[nstep: 563] loss: 0.008275524713099003
[step: 564] loss: 0.0008924180874601007
[nstep: 564] loss: 0.008206529542803764
[step: 565] loss: 0.0008910312317311764
[nstep: 565] loss: 0.00823259074240923
[step: 566] loss: 0.0008896990329958498
[nstep: 566] loss: 0.008255071938037872
[step: 567] loss: 0.0008884681737981737
[nstep: 567] loss: 0.008224990218877792
[step: 568] loss: 0.0008874415070749819
[nstep: 568] loss: 0.00820298120379448
[step: 569] loss: 0.0008868210134096444
[nstep: 569] loss: 0.008206544443964958
[step: 570] loss: 0.0008870346937328577
[nstep: 570] loss: 0.00820827390998602
[step: 571] loss: 0.0008889727760106325
[nst

[nstep: 673] loss: 0.007860967889428139
[step: 674] loss: 0.0007737959967926145
[nstep: 674] loss: 0.007858372293412685
[step: 675] loss: 0.0007757262792438269
[nstep: 675] loss: 0.007854428142309189
[step: 676] loss: 0.0007756563136354089
[nstep: 676] loss: 0.007851638831198215
[step: 677] loss: 0.0007719587301835418
[nstep: 677] loss: 0.007849069312214851
[step: 678] loss: 0.0007691369974054396
[nstep: 678] loss: 0.007846196182072163
[step: 679] loss: 0.0007691806531511247
[nstep: 679] loss: 0.00784494448453188
[step: 680] loss: 0.0007694101077504456
[nstep: 680] loss: 0.007845403626561165
[step: 681] loss: 0.0007676345994696021
[nstep: 681] loss: 0.007847772911190987
[step: 682] loss: 0.0007650199695490301
[nstep: 682] loss: 0.007855708710849285
[step: 683] loss: 0.0007635560468770564
[nstep: 683] loss: 0.007870812900364399
[step: 684] loss: 0.0007633026107214391
[nstep: 684] loss: 0.007895288988947868
[step: 685] loss: 0.0007628431776538491
[nstep: 685] loss: 0.007934505119919777
[

[step: 787] loss: 0.0006918180733919144
[nstep: 787] loss: 0.007529277354478836
[step: 788] loss: 0.0006851570797152817
[nstep: 788] loss: 0.007512989919632673
[step: 789] loss: 0.0006877685664221644
[nstep: 789] loss: 0.007500489242374897
[step: 790] loss: 0.0006878685671836138
[nstep: 790] loss: 0.0074928863905370235
[step: 791] loss: 0.0006820707931183279
[nstep: 791] loss: 0.007491079159080982
[step: 792] loss: 0.0006831229547969997
[nstep: 792] loss: 0.00749216927215457
[step: 793] loss: 0.000684195663779974
[nstep: 793] loss: 0.007492645177990198
[step: 794] loss: 0.0006795864901505411
[nstep: 794] loss: 0.007493751123547554
[step: 795] loss: 0.0006789896870031953
[nstep: 795] loss: 0.00749692739918828
[step: 796] loss: 0.0006802294519729912
[nstep: 796] loss: 0.007504208479076624
[step: 797] loss: 0.0006773778004571795
[nstep: 797] loss: 0.007513420190662146
[step: 798] loss: 0.0006756591610610485
[nstep: 798] loss: 0.007532837800681591
[step: 799] loss: 0.0006764737772755325
[n

[nstep: 901] loss: 0.007174004800617695
[step: 902] loss: 0.0006202742806635797
[nstep: 902] loss: 0.007171496283262968
[step: 903] loss: 0.0006133790011517704
[nstep: 903] loss: 0.007163562346249819
[step: 904] loss: 0.0006287592113949358
[nstep: 904] loss: 0.007155543193221092
[step: 905] loss: 0.0006177592440508306
[nstep: 905] loss: 0.007152894511818886
[step: 906] loss: 0.0006056653801351786
[nstep: 906] loss: 0.007153225131332874
[step: 907] loss: 0.0006155953160487115
[nstep: 907] loss: 0.007151838857680559
[step: 908] loss: 0.0006137414602562785
[nstep: 908] loss: 0.007149045355618
[step: 909] loss: 0.0006017704727128148
[nstep: 909] loss: 0.007148876786231995
[step: 910] loss: 0.0006058824947103858
[nstep: 910] loss: 0.0071525732055306435
[step: 911] loss: 0.0006092640687711537
[nstep: 911] loss: 0.007163806818425655
[step: 912] loss: 0.0006002035224810243
[nstep: 912] loss: 0.007185225374996662
[step: 913] loss: 0.0005993100930936635
[nstep: 913] loss: 0.007223629858344793
[s

In [2]:
# 원래데이터로 되돌리기
def ReturnToOriginal(data,data_max,data_min):
    #og = data*(data_max+1e-7)-data_min*(data-1)
    data_max = data_max+(1e-7)
    for i in range(0,len(data)):
        data[i,:]= (data[i,:]*data_max[i])-(data[i,:]-1)*data_min[i]
    return data



In [3]:
test_predict_to_original = ReturnToOriginal(test_predict,data_max[train_size+seq_length:],data_min[train_size+seq_length:])
testY_to_original = ReturnToOriginal(testY,data_max[train_size+seq_length:],data_min[train_size+seq_length:])

In [67]:
#json 파일로 만들기
import json
from collections import OrderedDict
file_data = OrderedDict()

day_time = 0

file_data["sigungu"] = {"Seoul_Jongno-gu":str(test_predict_to_original[:,0]),
                            "Seoul_Jung-gu":str(test_predict_to_original[:,1]),
                            "Seoul_Yongsan-gu":str(test_predict_to_original[:,2]),
                            "Seoul_Seongdong-gu":str(test_predict_to_original[:,3]),
                            "Seoul_Gwangjin-gu":str(test_predict_to_original[:,4]),
                            "Seoul_Dongdaemun-gu":str(test_predict_to_original[:,5]),
                            "Seoul_Jungnang-gu":str(test_predict_to_original[:,6]),
                            "Seoul_Seongbuk-gu":str(test_predict_to_original[:,7]),
                            "Seoul_Gangbuk-gu":str(test_predict_to_original[:,8]),
                            "Seoul_Dobong-gu":str(test_predict_to_original[:,9]),
                            "Seoul_Nowon-gu":str(test_predict_to_original[:,10]),
                            "Seoul_Eunpyeong-gu":str(test_predict_to_original[:,11]),
                            "Seoul_Seodaemun-gu":str(test_predict_to_original[:,12]),
                            "Seoul_Mapo-gu":str(test_predict_to_original[:,13]),
                            "Seoul_Yangcheon-gu":str(test_predict_to_original[:,14]),
                            "Seoul_Gangseo-gu":str(test_predict_to_original[:,15]),
                            "Seoul_Guro-gu":str(test_predict_to_original[:,16]),
                            "Seoul_Geumcheon-gu":str(test_predict_to_original[:,17]),
                            "Seoul_Yeongdeungpo-gu":str(test_predict_to_original[:,18]),
                            "Seoul_Dongjak-gu":str(test_predict_to_original[:,19]),
                            "Seoul_Gwanak-gu":str(test_predict_to_original[:,20]),
                            "Seoul_Seocho-gu":str(test_predict_to_original[:,21]),
                            "Seoul_Gangnam-gu":str(test_predict_to_original[:,22]),
                            "Seoul_Songpa-gu":str(test_predict_to_original[:,23]),
                            "Seoul_Gangdong-gu":str(test_predict_to_original[:,24])}


print(json.dumps(file_data,ensure_ascii =False,indent = "\t"))

{
	"sigungu": {
		"Seoul_Jongno-gu": "[ 8067.6895  8216.719   6043.035   5347.1055  3820.9094  3293.149\n  2495.1     3212.4004  3295.1328  4205.0215  4524.338   7686.716\n 11480.725  16184.938  16283.29   18204.732  17976.297  20918.969\n 18204.979  18019.928  15685.555  17692.756  15741.231  15578.643\n 14200.642  15789.244  14433.665  14706.007  12959.396  13065.931\n 11506.864  11658.321  10233.797  10249.063   8922.2705  9720.244\n  8863.919   9699.193   9208.809  11027.163  10449.341  12083.042\n 10901.754  12564.617  11354.58   13418.322  12430.914  13762.162\n 10422.312  10485.199   7969.461   6896.9805  5257.505   4671.9478\n  3660.773   4035.007   3968.3933  4594.192   4353.2227  5780.353\n  6722.915   9584.264  10934.264  14615.81   15582.412  17898.686\n 15954.615  16241.091  13999.97   15842.749  14546.552  15106.494\n 13344.145  13623.642  11926.414  12534.777  11166.13   11580.072\n 10287.557  11540.177  10373.832  11046.9     9666.647  10022.588\n  9064.313   9799.574  

In [68]:
with open("/Users/yeseo/Desktop/predict_"+str(2) +".json",'w',encoding = "utf-8") as make_file:
    json.dump(file_data,make_file,ensure_ascii =False,indent = "\t")